In [1]:
import pandas as pd
train_df=pd.read_csv('/content/train (7).csv')



In [2]:
train_df.describe()

,ID,AI
count,747.000000,746.000000
mean,493.480589,0.277480
std,286.582321,0.448056
min,1.000000,0.000000
25%,250.500000,0.000000
50%,485.000000,0.000000
75%,734.500000,1.000000
max,999.000000,1.000000


In [3]:
train_df.isna().sum()

ID        1
prompt    0
answer    1
AI        2
dtype: int64

In [4]:
train_df.dropna(inplace=True)



In [5]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df.answer.to_numpy(),
                                                                            train_df.AI.to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for r

In [7]:
import tensorflow as tf
import tensorflow_hub as hub

In [8]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 

In [9]:
model = tf.keras.Sequential([
  sentence_encoder_layer, 
  
  # take in sentences and then encode them into an embedding
  tf.keras.layers.Dense(128, activation="relu"),

  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
], name="model")

# Compile model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,871,809
Trainable params: 73,985
Non-trainable params: 256,797,824
_________________________________________________________________


In [10]:
import os

# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            save_best_only=True) # save only the best model to file

In [11]:
model_history = model.fit(train_sentences,
                              train_labels,
                              epochs=30,
                              validation_data=(val_sentences, val_labels),
                             callbacks=[create_model_checkpoint(model_name='kaggle')] )

Epoch 1/30
19/21 [==========================>...] - ETA: 0s - loss: 0.6191 - accuracy: 0.7253

21/21 [==============================] - 12s 479ms/step - loss: 0.6162 - accuracy: 0.7213 - val_loss: 0.5534 - val_accuracy: 0.7200
Epoch 2/30
20/21 [===========================>..] - ETA: 0s - loss: 0.4959 - accuracy: 0.7203

21/21 [==============================] - 8s 417ms/step - loss: 0.4866 - accuracy: 0.7273 - val_loss: 0.4503 - val_accuracy: 0.7467
Epoch 3/30
17/21 [=======================>......] - ETA: 0s - loss: 0.3910 - accuracy: 0.8290

21/21 [==============================] - 9s 457ms/step - loss: 0.3805 - accuracy: 0.8331 - val_loss: 0.3748 - val_accuracy: 0.8267
Epoch 4/30
18/21 [========================>.....] - ETA: 0s - loss: 0.3048 - accuracy: 0.8819

21/21 [==============================] - 10s 497ms/step - loss: 0.2948 - accuracy: 0.8867 - val_loss: 0.3317 - val_accuracy: 0.8267
Epoch 5/30
18/21 [========================>.....] - ETA: 0s - loss: 0.2310 - accuracy: 0.9149

21/21 [==============================] - 8s 414ms/step - loss: 0.2335 - accuracy: 0.9136 - val_loss: 0.3220 - val_accuracy: 0.8533
Epoch 6/30
21/21 [==============================] - 0s 15ms/step - loss: 0.1907 - accuracy: 0.9344 - val_loss: 0.3326 - val_accuracy: 0.8667
Epoch 7/30
21/21 [==============================] - 0s 15ms/step - loss: 0.1555 - accuracy: 0.9478 - val_loss: 0.3605 - val_accuracy: 0.8667
Epoch 8/30
21/21 [==============================] - 0s 15ms/step - loss: 0.1249 - accuracy: 0.9568 - val_loss: 0.3886 - val_accuracy: 0.8533
Epoch 9/30
21/21 [==============================] - 0s 15ms/step - loss: 0.1083 - accuracy: 0.9642 - val_loss: 0.3899 - val_accuracy: 0.8267
Epoch 10/30
21/21 [==============================] - 0s 17ms/step - loss: 0.0852 - accuracy: 0.9762 - val_loss: 0.4270 - val_accuracy: 0.8400
Epoch 11/30
21/21 [==============================] - 0s 15ms/step - loss: 0.0715 - accuracy: 0.9747 - val_loss: 0.4751 - val_accuracy: 0.8267
Epoch 12/30
21/21 [==

In [12]:
model.evaluate(val_sentences,val_labels)

3/3 [==============================] - 0s 10ms/step - loss: 0.6358 - accuracy: 0.8267


[0.6357718706130981, 0.8266666531562805]

In [13]:
model=tf.keras.models.load_model("/content/model_experiments/kaggle")
model.evaluate(val_sentences,val_labels)

3/3 [==============================] - 0s 10ms/step - loss: 0.3220 - accuracy: 0.8533


[0.3220413625240326, 0.8533333539962769]

In [14]:
test=pd.read_csv('/content/test (4).csv')
test.head()

,ID,prompt,answer
0,710,what's the best fast food,"As I mentioned earlier, ""best"" fast food is su..."
1,487,What is JSX in React?,JSX is a syntax extension for JavaScript that ...
2,136,What is the name of the reaction where a subst...,The reaction where a substance combines with o...
3,44,What is your opinion on the current state of t...,The current state of the environment is a caus...
4,627,What is deep learning?,Deep learning is a subfield of machine learnin...


In [15]:
subm=pd.read_csv('/content/sample_submission (4).csv')
subm.head()

,Id,Category
0,710,1
1,487,1
2,136,1
3,44,1
4,627,1


In [16]:
preds=model.predict(test.answer)

8/8 [==============================] - 0s 12ms/step


In [18]:
preds=tf.squeeze(tf.round(preds))
preds

<tf.Tensor: shape=(249,), dtype=float32, numpy=
array([0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0

In [19]:
preds=tf.cast(preds,tf.int32)
preds

<tf.Tensor: shape=(249,), dtype=int32, numpy=
array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0], dtype=int32)>

In [20]:
subm.Category=preds

In [21]:
subm.head()

,Id,Category
0,710,0
1,487,0
2,136,1
3,44,1
4,627,0


In [22]:
subm.Category.value_counts()

0    186
1     63
Name: Category, dtype: int64

In [ ]:
#and we download the 
from google.colab import files
subm.to_csv('subm.csv',index=False)
files.download('subm.csv')